In [ ]:
# Connecting to google drive.

import os
from google.colab import drive

# Define the base path for your Google Drive.
base_path = '/content/drive'

# Define the specific folder path within your Google Drive.
folder_path = 'MyDrive/Colab Notebooks/Crafting Tech'

# Combine the base path and folder path to create the full mount path.
full_project_path = os.path.join(base_path, folder_path)

# Mount your drive.
drive.mount(base_path, force_remount=True)

Mounted at /content/drive


In [ ]:
# Data handling and time parsing
import pandas as pd
from datetime import datetime
import os

# Define a function to identify government involvement.
def is_government(side):
  return side.str.contains("Government", case=False, na=False)

# Load the UCDP dyadic conflict dataset from the project directory.
file_name = "Dyadic_v24_1.csv"
file_path = os.path.join(full_project_path, file_name)
df = pd.read_csv(file_path)

# Convert the 'start_date2' column to datetime objects for timeline operations.
df['start_date2'] = pd.to_datetime(df['start_date2'], errors='coerce')
# Optionally convert 'start_date' if needed in future analysis
# df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')

# Get the most recent entry per dyad_id.
latest_conflicts = df.sort_values('start_date2').groupby('dyad_id').tail(1)

# Filter for ongoing conflicts by year.
latest_year = 2022 # df['year'].max()
ongoing_conflicts = latest_conflicts[latest_conflicts['year'] == latest_year]

# No government filtering needed.
result_conflicts = ongoing_conflicts.copy()

# Build timeline for all conflicts.
result_conflicts.loc[:, 'timeline'] = result_conflicts['start_date2'].dt.date.astype(str) + " to ongoing"

# Select desired columns.
result = result_conflicts[[
    'location',
    'territory_name',
    'type_of_conflict',
    'side_a',
    'side_b',
    'timeline',
    'intensity_level'
]]

# Rename for clarity.
result = result.rename(columns={'location': 'country'})

# Save to CSV.
output_file = "ucdp_ongoing_conflicts.csv"
output_path = os.path.join(full_project_path, output_file)
if os.path.exists(output_path):
    print(f"⚠️ File '{output_file}' already exists. Removing it.")
    os.remove(output_path)
result.to_csv(output_path, mode='w', index=False)
print(f"✅ Data saved to '{output_file}'")